In [2]:
import numpy as np
import pandas as pd
# import math
# from itertools import count
from tqdm.auto import tqdm
# import threading
import os, platform
# import multiprocessing
# import subprocess
# from queue import Empty as QueueEmpty
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from matplotlib import rc
rc('text',usetex=True)
rc('text.latex', preamble='\\usepackage{color}')

In [3]:
target_name = 'random2'
sym_graver_bin = '../build/sym_hilbert'

In [8]:
os.makedirs(f'mats_{target_name}', exist_ok=True)
problem_list = []
for n in range(100):
    out_fname = f'mats_{target_name}/problem{n:03}.mat'
    if not os.path.exists(out_fname):
        cmd = f'{sym_graver_bin} -unit-test-generate-matrix 5 15 {out_fname}'
        print(cmd)
        os.system(cmd)

    problem_list.append(out_fname.split('.')[0])

with open(f'models_{target_name}.txt', 'w') as f:
    for m in problem_list:
        f.write(m + '\n')